In [1]:
import stripe
import pandas as pd
import numpy as np
import psycopg2
from time import sleep
from config import api_key
from config import host, database, user, password
from queries import *
import pandas.io.sql as sqlio


In [2]:
def get_df():
    sub_df = pd.DataFrame()
    stripe.api_key = api_key
                                                        
    #first query to get response id
    first_response = stripe.Subscription.list(status='all')
    #creating dataframe with responses
    sub_df = sub_df.append(pd.json_normalize(first_response['data']))
    #defining next page for loops
    start_after = first_response['data'][-1]['id']
    print('Getting Results')

    while True:    
        try:
            #appending to the dataframe
            response = stripe.Subscription.list(starting_after=start_after, status='all')
            subscription_df = pd.json_normalize(response['data'])
            sub_df = sub_df.append(subscription_df, ignore_index=True)
            #giving the key for the next iteration
            start_after = response['data'][-1]['id']
            #ask george about rate limit issues
            sleep(1)
        except:
            print('Stripe Request went wrong')
        
        
        if response["has_more"] == False:
            break
    return sub_df


In [3]:
sub_df = get_df()

Getting Results


In [4]:
def cleaning_df(df):
    df = df[['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end', 'canceled_at', 'created', 'current_period_end', 'current_period_start', 'customer', 'ended_at', 'quantity', 'start_date', 'status']]
    df['billing_cycle_anchor'] = pd.to_datetime(sub_df['billing_cycle_anchor'], unit='s')
    df['canceled_at'] = pd.to_datetime(sub_df['canceled_at'], unit='s')
    df['cancel_at'] = pd.to_datetime(sub_df['cancel_at'], unit='s')
    df['created'] = pd.to_datetime(sub_df['created'], unit='s')
    df['current_period_end'] = pd.to_datetime(sub_df['current_period_end'], unit='s')
    df['current_period_start'] = pd.to_datetime(sub_df['current_period_start'], unit='s')
    df['ended_at'] = pd.to_datetime(sub_df['ended_at'], unit='s')
    df['start_date'] = pd.to_datetime(sub_df['start_date'], unit='s')
    df = df.replace({np.NaN: None})
    return df


In [5]:
sub_df = cleaning_df(sub_df)
sub_df

C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_17248/823688802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['billing_cycle_anchor'] = pd.to_datetime(sub_df['billing_cycle_anchor'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_17248/823688802.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['canceled_at'] = pd.to_datetime(sub_df['canceled_at'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_17248/823688802.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

,id,billing_cycle_anchor,cancel_at,cancel_at_period_end,canceled_at,created,current_period_end,current_period_start,customer,ended_at,quantity,start_date,status
0,sub_1L8hAPLnLiSMHjeFOk7Hzzf0,2022-06-09 08:56:37,None,False,None,2022-06-09 08:56:37,2022-07-09 08:56:37,2022-06-09 08:56:37,cus_Kq5dUDEXGqYNFX,None,1,2022-06-09 08:56:37,active
1,sub_1L8bGPLnLiSMHjeFCN2KrbKW,2022-06-09 02:38:25,None,False,None,2022-06-09 02:38:25,2022-07-09 02:38:25,2022-06-09 02:38:25,cus_LqFdFeIJQRC9be,None,1,2022-06-09 02:38:25,active
2,sub_1L8agULnLiSMHjeFhQcBmnmh,2022-06-09 02:01:18,None,False,None,2022-06-09 02:01:18,2022-07-09 02:01:18,2022-06-09 02:01:18,cus_Lma7BlsNPcylAP,None,1,2022-06-09 02:01:18,active
3,sub_1L8Y24LnLiSMHjeFnJm7oAZ7,2022-06-08 23:11:24,None,False,None,2022-06-08 23:11:24,2022-07-08 23:11:24,2022-06-08 23:11:24,cus_LdsM14Xsj9KT48,None,1,2022-06-08 23:11:24,active
4,sub_1L8V3cLnLiSMHjeFfbKghszO,2022-06-08 20:00:48,None,False,None,2022-06-08 20:00:48,2022-07-08 20:00:48,2022-06-08 20:00:48,cus_LqBPgyw48imqA0,None,1,2022-06-08 20:00:48,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,sub_HxRsSFzSCdOCao,2020-09-04 04:57:19,None,False,2020-12-21 20:49:31,2020-09-04 04:57:19,2021-01-04 04:57:19,2020-12-04 04:57:19,cus_HsxJ4mZ2mIvuaw,2020-12-21 20:49:31,1,2020-09-04 04:57:19,canceled
1268,sub_Hx4l1OOH4jFpCg,2020-09-03 05:03:49,None,False,2020-11-23 22:57:12,2020-09-03 05:03:49,2020-12-03 05:03:49,2020-11-03 05:03:49,cus_HsRzIRAfM5a8Pt,2020-11-23 22:57:12,1,2020-09-03 05:03:49,canceled
1269,sub_Hx4hMQdjs9NK8q,2020-09-03 05:00:03,None,False,2020-09-03 05:02:56,2020-09-03 05:00:03,2020-10-03 05:00:03,2020-09-03 05:00:03,cus_HsRzIRAfM5a8Pt,2020-09-03 05:02:56,1,2020-09-03 05:00:03,canceled
1270,sub_Hww73rW1qJJn9n,2020-09-02 20:07:40,None,False,2020-09-02 21:36:40,2020-09-02 20:07:40,2020-10-02 20:07:40,2020-09-02 20:07:40,cus_HsRzIRAfM5a8Pt,2020-09-02 21:36:40,1,2020-09-02 20:07:40,canceled


In [6]:
def connect(host, database, user, password):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host, database=database, user=user, password=password)
        conn.autocommit = True
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(host, database, user, password)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
def setup(conn, schema, table):
    cur = conn.cursor()
    cur.execute(schema)
    cur.execute(table)
    

In [8]:
setup(conn, create_subscription_schema, create_subscription_table)

In [9]:
def insert_values(conn, df, table):
    
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    cur = conn.cursor()
    values = [cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tup).decode('utf8') for tup in tuples]
    query  = (f'INSERT INTO {table(cols)} VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT({values[0]}) DO UPDATE SET')
    
    try:
         cur.execute(query, tuples)
         conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1
    print("execute_mogrify() done")
    cursor.close()
   

In [10]:
#insert_values(conn, sub_df, 'subscriptions')

In [11]:
sub_df.columns

Index(['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end',
       'canceled_at', 'created', 'current_period_end', 'current_period_start',
       'customer', 'ended_at', 'quantity', 'start_date', 'status'],
      dtype='object')

In [12]:
def insert_values_v2(conn, df):
    
    cur = conn.cursor()
    
    for value in df.values:
        subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at, quantity, start_date, status = value
        print(value)
        #insert subscriber record
        subscriber_data = (subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at,quantity, start_date, status)
        cur.execute(insert_table, subscriber_data)
    



In [13]:
insert_values_v2(conn, sub_df)

['sub_1L894ZLnLiSMHjeFr8J3N999' Timestamp('2022-06-07 20:32:19') None
 False None Timestamp('2022-06-07 20:32:19')
 Timestamp('2022-07-07 20:32:19') Timestamp('2022-06-07 20:32:19')
 'cus_Lpogo5G2OilRxD' None 1 Timestamp('2022-06-07 20:32:19') 'active']


NameError: name 'insert_table' is not defined

In [ ]:
cur = conn.cursor()
sql = '''SELECT * FROM subscriptions'''
  
  
cur.execute(sql)
column_names = [desc[0] for desc in cur.description]
for i in column_names:
    print(i)

In [ ]:
sub_df

In [ ]:
cur.execute('SELECT * FROM subscriptions')

In [ ]:
print(cur.fetchall())

In [8]:
test_df_4_me = sqlio.read_sql_query("SELECT * FROM subscriptions", conn)
test_df_4_me

,subscription_id,billing_cycle_anchor,cancel_at,cancel_at_period_end,canceled_at,created,current_period_end,current_period_start,customer_id,ended_at,quantity,start_date,status
0,sub_1L3Po0LnLiSMHjeFGEURBARU,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_Lkvdmuz0VmjG29,None,1,2022-05-25,active
1,sub_1L3Mk9LnLiSMHjeFB74ZFDBB,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_LkphyJOTDuBa6e,None,1,2022-05-25,active
2,sub_1L3DQULnLiSMHjeFR8Ky7bTn,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_LKj6mDzeUgI8l7,None,1,2022-05-25,active
3,sub_1L3DMwLnLiSMHjeF0Ma0xP40,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_Lkil0CHKttza9s,None,1,2022-05-25,active
4,sub_1L3BJCLnLiSMHjeFdcn3bqz6,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_LkgZyDBSqAJuzr,None,1,2022-05-25,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,sub_1L8HD2LnLiSMHjeF0Lys4V9p,2022-06-08,None,False,None,2022-06-08,2022-07-08,2022-06-08,cus_K9IEgMmlt35AgP,None,1,2022-06-08,active
1268,sub_1L8FoiLnLiSMHjeFwPyjy0Ug,2022-06-08,None,False,None,2022-06-08,2022-07-08,2022-06-08,cus_LpB0T29KUyj6tC,None,1,2022-06-08,active
1269,sub_1L8FZrLnLiSMHjeFd6fECVD2,2022-06-08,None,False,None,2022-06-08,2022-07-08,2022-06-08,cus_LpPX4kjd6w1TZk,None,1,2022-06-08,active
1270,sub_1L894ZLnLiSMHjeFr8J3N999,2022-06-07,None,False,None,2022-06-07,2022-07-07,2022-06-07,cus_Lpogo5G2OilRxD,None,1,2022-06-07,active


In [ ]:
conn.close()